In [ ]:
'MY ATTEMPT AT IMPLEMENTING Differentiable Neural Computer'

In [ ]:
'IMPORT LIBRARIES'

import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random

tf.compat.v1.set_random_seed(10)
tf.compat.v1.reset_default_graph()


In [ ]:
'HELPER METHODS'

output_dim = 4
hid_dim=32
N = 10
W = 10
R = 1
input_features = 50

    
def oneplus(x):
    return 1 + tf.nn.softplus(x)
    
sizes = [R * W, R, W, 1, W, W, R, 1, 1]
names = ["r_read_keys", "r_read_strengths", "write_key", "write_strength", "erase_vector", "write_vector",
         "r_free_gates", "allocation_gate", "write_gate"]
functions = [tf.identity, oneplus, tf.identity, oneplus, tf.nn.sigmoid, tf.identity,
             tf.nn.sigmoid, tf.nn.sigmoid, tf.nn.sigmoid]
indexes = [[sum(sizes[:i]), sum(sizes[:i + 1])] for i in range(len(sizes))]
assert len(names) == len(sizes) == len(functions) == len(indexes)
# Computing the interface dictionary from ordered lists of sizes, vector names and functions, as in the paper
# self.split_interface is called later with the actual computed interface vector
split_interface = lambda iv: {name: fn(iv[:, i[0]:i[1]]) for name, i, fn in
                                   zip(names, indexes, functions)}

epsilon = 1e-6
epsilon = int(epsilon)

def content_lookup(memory, keys, strength):
    """

    :param memory: array of shape [batch_size, memory_size, word_size], i.e. [b, n, w]
    :param keys: array of shape [batch_size, n_keys, word_size], i.e. [b, r, w]
    :param strength: array of shape [batch_size, n_keys], i.e. [b, r]
    :return: tensor of shape [batch_size, memory_size, n_keys]
    """
    keys = tf.nn.l2_normalize(keys, axis=2)
    memory = tf.nn.l2_normalize(memory, axis=2)
    similarity = tf.einsum("bnw,brw,br->bnr", memory, keys, strength)
    content_weighting = tf.nn.softmax(similarity, axis=1, name="Content_weighting")

    return content_weighting

def calculate_allocation_weighting(usage_vec):
    usage_vector = epsilon + (1 - epsilon) * usage_vec

    highest_usage, inverse_indices = tf.nn.top_k(-usage_vector, k=N*W)
    lowest_usage = -highest_usage

    allocation_scrambled = (1 - lowest_usage) * tf.math.cumprod(lowest_usage, axis=1, exclusive=True)

    # allocation is not in the correct order. alloation[i] contains the sorted[i] value
    # reversing the already inversed indices for each batch
    indices = tf.stack([tf.math.invert_permutation(batch_indices) for batch_indices in tf.unstack(inverse_indices)])
    allocation = tf.stack([tf.gather(mem, ind)
                           for mem, ind in
                           zip(tf.unstack(allocation_scrambled), tf.unstack(indices))])

    return allocation

In [ ]:
'THE DNC'

epsilon = 1e-6
epsilon = int(epsilon)   
class DNC: 
    def __init__(self, x, N, W, R, batch_size):
      with tf.device('/GPU:0'):
        self.x = x
        print('X: ', self.x.shape)
        self.N = N
        print('N: ', self.N)
        self.W = W
        print('W: ', self.W)
        self.R = R
        print('R: ', self.R)
        self.batch_size = batch_size
        print('batch_size: ', self.batch_size)
        self.memory_size = self.N*self.W
        print('MEMORY SIZE: ', self.memory_size)
        

        self.memory = tf.fill([self.batch_size, self.N*self.W, self.W], epsilon)
        print('MEMORY: ', self.memory)
        self.memory = tf.cast(self.memory, tf.float32)

        self.read_vectors = tf.fill([self.batch_size, self.R*self.W], epsilon)
        self.read_vectors = tf.cast(self.read_vectors, tf.float32)
        print('READ VECTORS: ', self.read_vectors.shape)
        

        self.input_dim = self.read_vectors.shape[-1] + input_features 
        print('FEATURES: ', input_features) 
        print('INPUT DIM: ', self.input_dim) #CONTROLLER INPUT DIMENSION IS SUM OF INPUT FEATURES AND READ VECTOR SIZE

        # HIDDEN
        self.w1 = tf.Variable(tf.compat.v1.random_normal([self.input_dim, hid_dim]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.compat.v1.random_normal([hid_dim]), dtype=tf.float32)

        # OUT
        self.w2 = tf.Variable(tf.compat.v1.random_normal([hid_dim, output_dim]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.compat.v1.random_normal([output_dim]), dtype=tf.float32)

        self.interface_vec_size = (self.W*self.R) + 3*self.W + 5*self.R + 3
        self.interface_weights = tf.Variable(tf.compat.v1.random_normal([hid_dim, self.interface_vec_size]), dtype=tf.float32)
        print('INTERFACE WEIGHTS: ', self.interface_weights.shape)

        self.memory_weights = tf.Variable(tf.compat.v1.random_normal([self.R* self.W, output_dim]))
        print('MEMORY WEIGHTS: ', self.memory_weights.shape)

        #initial interface weightings
        self.write_weighting = tf.fill([self.batch_size, self.N*self.W], epsilon, name="Write_weighting")
        self.write_weighting = tf.cast(self.write_weighting, tf.float32)

        self.usage_vector = tf.zeros([self.batch_size, self.N*self.W], name="Usage_vector")

        self.read_weightings = tf.fill([self.batch_size, self.N*self.W, self.R], epsilon)
        self.read_weightings = tf.cast(self.read_weightings, tf.float32)
        
        self.sizes = [self.R * self.W, self.R, self.W, 1, self.W, self.W, self.R, 1, 1]
        self.names = ["r_read_keys", "r_read_strengths", "write_key", "write_strength", "erase_vector", "write_vector",
                 "r_free_gates", "allocation_gate", "write_gate"]
        self.functions = [tf.identity, oneplus, tf.identity, oneplus, tf.nn.sigmoid, tf.identity,
                     tf.nn.sigmoid, tf.nn.sigmoid, tf.nn.sigmoid]

        self.indexes = [[sum(self.sizes[:i]), sum(self.sizes[:i + 1])] for i in range(len(self.sizes))]
        assert len(self.names) == len(self.sizes) == len(self.functions) == len(self.indexes)
        self.split_interface = lambda iv: {name: fn(iv[:, i[0]:i[1]]) for name, i, fn in
                                           zip(self.names, self.indexes, self.functions)}

        
    
    def step(self): 
      with tf.device('/GPU:0'):
        global memory, write_weighting 
        self.x = tf.concat([self.x, self.read_vectors], axis=1) 
        print('X AFTER CONCATENATION: ', self.x.shape)
        self.h1 = tf.nn.relu(tf.matmul(self.x, self.w1) + self.b1)
        print('H1: ', self.h1.shape)
        self.controller_out = tf.matmul(self.h1, self.w2) + self.b2
        print('CONTROLLER OUTPUT: ', self.controller_out.shape)
        self.interface_out = tf.matmul(self.h1, self.interface_weights)
        print('INTERFACE OUTPUT: ', self.interface_out.shape)
        self.intf = self.split_interface(self.interface_out)
        #reshape keys
        print('WRITE KEY: ', self.intf['write_key'].shape)
        self.intf["write_key"] = tf.expand_dims(self.intf["write_key"], axis=1)
        print('WRITE KEY: ', self.intf['write_key'].shape)
        self.intf["r_read_keys"] = tf.reshape(self.intf["r_read_keys"], [-1, R, W])
        #WRITE
        self.write_content = content_lookup(self.memory, self.intf['write_key'], self.intf['write_strength'])
        print('WRITE CONTENT: ', self.write_content.shape)
        print('ALLOC GATE: ', self.intf["allocation_gate"].shape)
        # dynamic allocation
        self.memory_retention = tf.reduce_prod(1 - tf.einsum("br,bnr->bnr", self.intf["r_free_gates"], self.read_weightings), 2)
        self.usage_vector = (self.usage_vector + self.write_weighting - self.usage_vector * 
                             self.write_weighting) * self.memory_retention

        self.write_allocation = calculate_allocation_weighting(self.usage_vector)
        #final write calculations
        print('WRITE WEIGHTING: ', self.write_weighting.shape)
        print('WRITE ALLOCATION: ', self.write_allocation.shape)
        print('WRITE GATE: ', self.intf['write_gate'].shape)
        self.write_weighting = self.intf["write_gate"] * (self.intf["allocation_gate"] * self.write_allocation
                                                      + tf.einsum("bnr,bi->bn",
                                                                  self.write_content,
                                                                  (1 - self.intf["allocation_gate"])))
        
        self.memory = self.memory * (1 - tf.einsum("bn,bw->bnw", self.write_weighting, self.intf["erase_vector"])) + \
                    tf.einsum("bn,bw->bnw", self.write_weighting, self.intf["write_vector"])

        print(self.memory)
        #
        #READ 
        
        self.read_content_lookup = content_lookup(self.memory, self.intf['r_read_keys'], self.intf['r_read_strengths'])
        print('READ LOOK-UP WEIGHTS: ', self.read_content_lookup.shape)
        self.read_weightings = self.read_content_lookup
        print('READ WEIGHTING: ', self.read_weightings.shape)
        self.current_read_vectors = tf.einsum("bnw,bnr->brw", self.memory, self.read_weightings)
        self.current_read_vectors = tf.reshape(self.current_read_vectors, (-1, self.W * self.R))
        print('READ VECTORS: ', self.current_read_vectors.shape)
        self.memory_output = tf.matmul(self.current_read_vectors, self.memory_weights)
        print('MEMORY OUTPUT: ', self.memory_output.shape)

        self.read_vectors = self.current_read_vectors
        
        #bypass dropout
        #self.controller_out = tf.nn.dropout(self.controller_out, rate=0.5)

        self.final_out = self.controller_out + self.memory_output  
        print('FINAL OUTPUT: ', self.final_out.shape)
        print(self.final_out)
        return self.final_out



In [ ]:
'INSTANTIATE DNC'
batch_size = 1
features = 50

input_dim_init = 50

tf.debugging.set_log_device_placement(True)

with tf.device('/GPU:0'):

  x = tf.compat.v1.placeholder(tf.float32, (batch_size, input_dim_init), name='x')
  print('X: ', x.shape)
  y = tf.compat.v1.placeholder(tf.int32, (batch_size, ), name='y')
  print('Y: ', y.shape)

  a = DNC(x, N, W, R, batch_size)

  out = a.step()
  loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=out, labels=y))
  optimizer = tf.compat.v1.train.AdamOptimizer(0.001).minimize(loss)
  pred = tf.nn.softmax(out)

  print(pred.shape)

  correct_pred = tf.equal(tf.argmax(tf.nn.softmax(out), 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))